In [149]:
from xml.etree.ElementTree import ElementTree,Element
import os
import re
import numpy as np
import pandas as pd
from pandas.tseries.offsets import *
import math
import folium
import folium.plugins as plugins
import time
import webbrowser as web
import branca
from ipywidgets import interact
from ctmnet.simulation import Simulator
from ctmnet.networkcreation import create_from_xml

def read_xml(in_path):
    tree = ElementTree()
    tree.parse(in_path)
    return tree

def find_nodes(tree, path):
    return tree.findall(path)

def if_match(node, kv_map):
    for key in kv_map:
        if node.get(key) != kv_map.get(key):
            return False
    return True

def get_node_by_keyvalue(nodelist, kv_map):
    result_nodes = []
    for node in nodelist:
        if if_match(node, kv_map):
            result_nodes.append(node)
    return result_nodes

def show_value_by_path(nodelist, tag, kv_map):
    rlt = []
    for parent_node in nodelist:
        children = list(parent_node)
        if children[0].text == kv_map:
            for child in children:
                print(child.tag+"  "+child.text)
                
def cal_distance(x1, x2):
    x1 = tuple(map(math.radians, x1))
    x2 = tuple(map(math.radians, x2))
    r = 6378.137*1000
    a = x1[1]-x2[1]
    b = x1[0]-x2[0]
    temp = math.pow(math.sin(a/2), 2)+math.pow(math.sin(b/2),
                                               2)*math.cos(x1[1])*math.cos(x2[1])
    return 2*math.asin(math.sqrt(temp))*r


def line_split(x1, x2, inter_d):
    total_length = cal_distance(x1, x2)
    split_n = int(total_length/inter_d+0.5)
    points = []
    if split_n >= 1:
        for i in range(1, split_n):
            points.append((x1[0]+(x2[0]-x1[0])*i/split_n,
                           x1[1]+(x2[1]-x1[1])*i/split_n))
        return points
    else:
        print('两点之间距离小于分割长度')
        points.append(((x1[0]+x2[0])/2, (x1[1]+x2[1])/2))
        return points

def normalize(array):
    return (array-array.min())/(array.max()-array.min())
        
if __name__ == "__main__":
    
    sim = Simulator()
    sec = create_from_xml('net1.xml')
    
    #读取xml
    tree = read_xml("net1.xml")
    tree2 = read_xml("net.xml")
    
    up=[]
    down=[]
    
    #筛选上、下游
    c_nodes = find_nodes(tree, "connections/connector")
    for i in range(0,27):
        parent_node = get_node_by_keyvalue(c_nodes, {"id": str(i)})
        children = list(parent_node)[0]
        up.extend(re.findall(r'\d+',children[0].text))
        down.append(children[1].text)

    up=list(set(up))
    down=list(set(down))
    up=[int(i) for i in up]
    down=[int(i) for i in down]
    up.sort()
    down.sort()
    for u in up:
        if (u in down):
            up.remove(u)
            down.remove(u)
    
    print(up)
    print(down)
    
    #s_nodes = find_nodes(tree, "sections/section")
    for u in up:
        #parent_node = get_node_by_keyvalue(s_nodes, {"id": str(u)})
        #children = list(parent_node)[0]
        #print(children[0].text)
        U={'section_id':str(u),'volume':1750*3}
        #print(U)
        #sim.set_arrival(children[0].text,1750*0.6*3)
        sec.create('arrival',u,U)
    for d in down:
        #parent_node = get_node_by_keyvalue(s_nodes, {"id": str(d)})
        #children = list(parent_node)[0]
        #print(children[0].text)
        D={'section_id':str(d),'capacity':1750*3}
        #sim.set_depature(children[0].text,1750*3)
        sec.create('departure',d,D)


    #sim.set_arrival('诚信大道',1750*0.6*3)
    #sim.set_depature('诚信大道',1750*3)

    result = []
    for i in range(20):
        sec.run_single_step()
        S=sec.sections
        t=[]
        for i in range(48):
            t.append(S[str(i)].last_sim_step_volume())
        #result.append(eval(str(sec)))
        if i>=10:
            result.append(t)
    #print(result[8])
    
    s_nodes = find_nodes(tree, "sections/section")
    s_nodes2 = find_nodes(tree2, "sections/section")
    reloc=[]
    for i in range(48):
        parent_node = get_node_by_keyvalue(s_nodes, {"id": str(i)})
        children = list(parent_node)[0]
        c=re.findall(r'\d+',children[7].text)
        reloc.append([children[0].text,int(c[0]),int(c[1])])
    print(reloc)
    
    new_coors = []
    for r in reloc:
        for s in s_nodes2:
            if s.getchildren()[0].text==r[0]:
                co=re.findall(r'\d+.\d+, \d+.\d+',s.getchildren()[7].text)
                coors=[]
                for c in co[min(r[1],r[2]):max(r[1],r[2])+1]:
                    c=[float(c.split(",")[0]),float(c.split(",")[1])]
                    coors.append(c)
                coors.sort(key=lambda x: x[0])
                
                for x1, x2 in zip(coors[:-1], coors[1:]):
                    new_coors.extend(line_split(x1, x2, 50))
        
    #print(new_coors)
    print(len(new_coors),len(result[0]))
    data = []
    df = pd.DataFrame(new_coors, columns=['lon', 'lat'])
    
    for i in label:
        df['volume']=normalize(np.array(result[i]))
        data.append(df.values.tolist())
        
    '''''coors=
    
    coors.sort(key=lambda x: x[0])

    new_coors = []
    for x1, x2 in zip(coors[:-1], coors[1:]):
        new_coors.extend(line_split(x1, x2, 50))

    label = [i for i in range(len(result))]
    
    print(len(new_coors),len(result[0]))
    data = []
    df = pd.DataFrame(new_coors, columns=['lon', 'lat'])
    
    for i in label:
        df['volume']=normalize(np.array(result[i]))
        data.append(df.values.tolist())'''

[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 13, 14, 16, 18, 19, 21, 23, 27, 28, 31, 33, 34, 36, 39, 41, 43, 46, 47]
[8, 12, 15, 20, 22, 29, 30, 35, 37, 38, 40, 42, 44, 45]
[['诚信大道', 1, 2], ['诚信大道', 2, 4], ['诚信大道', 4, 7], ['诚信大道', 7, 9], ['诚信大道', 9, 11], ['水阁路', 33, 34], ['水阁路', 34, 39], ['水阁路', 39, 40], ['水阁路', 40, 9], ['佛城西路', 41, 42], ['佛城西路', 42, 43], ['佛城西路', 43, 45], ['金智路', 41, 11], ['水阁路', 9, 42], ['宁丹路', 49, 58], ['宁丹路', 58, 60], ['31路', 58, 68], ['31路', 68, 77], ['丹阳大道', 256, 339], ['丹阳大道', 339, 342], ['丹阳大道', 342, 2], ['丹阳大道', 2, 347], ['丹阳大道', 347, 45], ['40路', 58, 361], ['42路', 362, 347], ['44路', 363, 342], ['46路', 364, 339], ['46路', 339, 367], ['46路', 367, 373], ['46路', 373, 39], ['46路', 39, 11], ['宁丹路', 45, 361], ['佛城西路', 45, 60], ['丹阳大道', 60, 77], ['丹阳大道', 77, 362], ['丹阳大道', 362, 1], ['丹阳大道', 1, 363], ['丹阳大道', 363, 364], ['丹阳大道', 364, 256], ['65路', 492, 43], ['65路', 43, 7], ['65路', 7, 500], ['65路', 500, 373], ['73路', 4, 501], ['73路', 501, 367], ['73路', 367, 34], ['79路', 40, 500], [

d:\python\lib\site-packages\ipykernel_launcher.py:154: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
d:\python\lib\site-packages\ipykernel_launcher.py:155: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


ValueError: operands could not be broadcast together with shapes (48,) (47,) 

AttributeError: 'list' object has no attribute 'split'

In [43]:
s_nodes = find_nodes(tree, "sections/section")
for u in up:
    parent_node = get_node_by_keyvalue(s_nodes, {"id": str(u)})
    children = list(parent_node)[0]
    print(children[0].text)
    #sim.set_arrival(children[0].text,1750*0.6*3)

print(" ")
for d in down:
    parent_node = get_node_by_keyvalue(s_nodes, {"id": str(d)})
    children = list(parent_node)[0]
    print(children[0].text)
    #sim.set_depature(children[0].text,1750*3)

诚信大道
诚信大道
诚信大道
诚信大道
水阁路
水阁路
水阁路
佛城西路
佛城西路
佛城西路
水阁路
宁丹路
31路
丹阳大道
丹阳大道
丹阳大道
40路
46路
46路
宁丹路
丹阳大道
丹阳大道
丹阳大道
65路
65路
73路
79路
79路
 
水阁路
金智路
宁丹路
丹阳大道
丹阳大道
46路
46路
丹阳大道
丹阳大道
丹阳大道
65路
65路
73路
73路


In [59]:
S=sec.sections
for i in range(48):
    print(S[str(i)].last_sim_step_volume())
    #print(S[str(i)].inflow)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[5.241180371450532, 3.863587869313003, 2.7864185930474505, 1.760693895592008, 0.949584533710126, 0.436379490831885, 0.1724675677037069, 0.05937507040886597, 0.018029741529946373, 0.004882384578738977, 0.0011900389441248428, 0.00026312284888997954, 5.312229264149874e-05, 9.847924001299184e-06, 1.6844178285502076e-06, 2.6693280478084074e-07, 3.9335182550752244e-08, 5.407237353797142e-09, 6.953604589210428e-10, 8.38631243194947e-11, 9.506594140272784e-12, 1.0149254980077153e-12, 1.0222811616146637e-13, 9.730285669280932e-15, 8.764389576631326e-16, 7.48028525340182e-17, 6.056464301403678e-18, 4.656709993019445e-19, 3.4033622099175753e-20, 2.3663323323459925e-21, 1.566444233827274e-22, 9.879282788981617e-24, 5.939859024421551e-25, 3.406494734867253e-26, 

In [41]:
sec.last_sim_step_volume()

AttributeError: 'Simulator' object has no attribute 'last_sim_step_volume'

In [42]:
sec.last_sim_step_volume()

AttributeError: 'Simulator' object has no attribute 'last_sim_step_volume'

In [51]:
print(U)

{'section_id': '47', 'volume': 3150.0}
